[View in Colaboratory](https://colab.research.google.com/github/stanimman/Pneumonia-Detection/blob/master/Classification_Custom.ipynb)

In [0]:
!pip uninstall Pillow -y

!pip install Pillow

Uninstalling Pillow-4.0.0:
  Successfully uninstalled Pillow-4.0.0
    100% |████████████████████████████████| 2.0MB 8.3MB/s 


In [0]:
# install dependencies not included by Colab# insta 
# use pip3 to ensure compatibility w/ Google Deep Learning Images 
!pip3 install -q pydicom 
!pip3 install -q tqdm 
!pip3 install -q imgaug

In [0]:
import os 
import sys

import random
import math
import numpy as np
import cv2
import matplotlib.pyplot as plt
import json
import pydicom
from imgaug import augmenters as iaa
from tqdm import tqdm
import pandas as pd 
import glob

In [0]:
!pip install kaggle-cli
!kg download -u 'stanimman' -p '***' -c 'rsna-pneumonia-detection-challenge'

    100% |████████████████████████████████| 81kB 2.5MB/s 
    100% |████████████████████████████████| 5.3MB 4.1MB/s 
    100% |████████████████████████████████| 102kB 23.9MB/s 
    100% |████████████████████████████████| 81kB 19.7MB/s 
  Running setup.py bdist_wheel for kaggle-cli ... - \ done
  Stored in directory: /root/.cache/pip/wheels/d5/bb/10/c1dd1b08c7433c943cb55c46367ae3f891415e8a37300ff8a7
  Running setup.py bdist_wheel for cliff ... - \ done
  Stored in directory: /root/.cache/pip/wheels/02/22/09/66f8c243f9c68dee7e6456a0fd6c117439a64394fdaf02d965
  Running setup.py bdist_wheel for configparser ... - done
  Stored in directory: /root/.cache/pip/wheels/a3/61/79/424ef897a2f3b14684a7de5d89e8600b460b89663e6ce9d17c
  Running setup.py bdist_wheel for PrettyTable ... - done
  Stored in directory: /root/.cache/pip/wheels/80/34/1c/3967380d9676d162cb59513bd9dc862d0584e045a162095606
  Running setup.py bdist_wheel for pyperclip ... - done
  Stored in directory: /root/.cache/

In [0]:
!git clone https://github.com/stanimman/Pneumonia-Detection.git

Cloning into 'Pneumonia-Detection'...
remote: Counting objects: 31, done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 31 (delta 15), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (31/31), done.


In [0]:
!unzip -q -o stage_1_train_images.zip -d stage_1_train_images # Need to understand what is the various argument that is passed on

In [0]:
import PIL
print(PIL.PILLOW_VERSION)
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl torchvision
import torch
print(torch.__version__)
print(torch.cuda.is_available())

import torch 
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms

from torch.utils.data import DataLoader, Dataset
from torchvision.datasets.folder import pil_loader
from torchvision import transforms, utils
import pandas as pd
import matplotlib.pyplot as plt
from skimage import io, transform
import matplotlib.pyplot as plt
import os
from torchvision import  models
import torch.nn as nn
import torch.optim as optim
import copy
from torch.optim import lr_scheduler
import time
import datetime as dt
from PIL import Image
import cv2

5.2.0
tcmalloc: large alloc 1073750016 bytes == 0x5c488000 @  0x7fe4b7a521c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8
0.4.0
True


In [0]:
stage_1_train_labels_df = pd.read_csv('Pneumonia-Detection/stage_1_train_labels.csv')

In [0]:
import random
stage_1_train_labels_df['path'] = stage_1_train_labels_df['patientId']+'.dcm'
random_number_list = random.sample(range(0, 28988), 5000)
stage_1_test_labels_df = stage_1_train_labels_df.iloc[random_number_list,:]


In [0]:
Pnumenoia_Case = stage_1_train_labels_df[stage_1_train_labels_df.Target == 1]
Non_Pnumenoia_Case = stage_1_train_labels_df[stage_1_train_labels_df.Target == 0]
stage_1_train_labels_df = pd.concat([Pnumenoia_Case,(Non_Pnumenoia_Case.iloc[range(10000),:])])

In [0]:
stage_1_train_labels_df.Target.value_counts()

0    10000
1     8964
Name: Target, dtype: int64

In [0]:
Pnumenoia_Case_test = stage_1_test_labels_df[stage_1_test_labels_df.Target == 1]
Non_Pnumenoia_Case_test = stage_1_test_labels_df[stage_1_test_labels_df.Target == 0]
stage_1_test_labels_df = pd.concat([Pnumenoia_Case_test.iloc[range(1000),:],Non_Pnumenoia_Case_test.iloc[range(1000),:]])

In [0]:
stage_1_test_labels_df.Target.value_counts()

1    1000
0    1000
Name: Target, dtype: int64

In [0]:
Non_Pnumenoia_Case_test = stage_1_test_labels_df[(stage_1_test_labels_df.Target == 0)&(stage_1_test_labels_df.)]

In [0]:

stage_1_test_labels_df = pd.concat([Pnumenoia_Case_test.iloc[range(1000),:],Non_Pnumenoia_Case_test.iloc[range(1000),:]])

In [0]:
class RSNADataset(Dataset):
    """Mura dataset."""
    def __init__(self, df, root_dir, transform=None):
        """
        Args:
            df (dataframe): Path to the image file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.df = df
        self.root_dir = root_dir
        self.transform = transform
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir,
                                self.df.iloc[idx, 6])
        
        # Reading Pydicom Files 
        ds = pydicom.dcmread(img_name)
        image = ds.pixel_array
        image = np.stack((image,)*3,-1)
        labels = self.df.iloc[idx, 5]
        labels = labels.astype('float')
        if self.transform:
            image = self.transform(image)
        return [image, labels]

In [0]:
transformed_train_dataset = RSNADataset(df=stage_1_train_labels_df,
                                    root_dir='stage_1_train_images',
                                    transform=transforms.Compose([
                                               transforms.ToPILImage(),
                                               #transforms.RandomRotation(10),
                                               transforms.Resize(224),
                                               #transforms.CenterCrop(224),
                                               #transforms.RandomHorizontalFlip(),
                                               transforms.ToTensor(),
                                               transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                               
                                           ]))

transformed_valid_dataset = RSNADataset(df=stage_1_test_labels_df,
                                     root_dir='stage_1_train_images',
                                     transform=transforms.Compose([
                                                transforms.ToPILImage(),
                                                #transforms.RandomRotation(10),
                                                transforms.Resize(224),
                                                #transforms.CenterCrop(224),
                                                #transforms.RandomHorizontalFlip(),
                                                transforms.ToTensor(),
                                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                               
                                            ]))

In [0]:
transformed_train_dl = DataLoader(transformed_train_dataset,batch_size=48,shuffle=True)
transformed_valid_dl = DataLoader(transformed_valid_dataset,batch_size=48,shuffle=True)

In [0]:
def get_count(layer_name,model_name):
  ct  = 0 
  layer_count ={}
  for name, child in model_name.named_children():
      for name2, params in child.named_parameters():
        layer_count[name2] = ct
        ct +=1
  return layer_count[layer_name]

def freeze_till(layer_name,model_name):
  ct  = 0 
  count = get_count(layer_name,model_name)
  
  for name, child in model_name.named_children():
    for name2, params in child.named_parameters():
      
      if ct > count :
          
            params.requires_grad = True
      else :
            params.requires_grad = False
      ct +=1

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

dataloaders = {"train": transformed_train_dl, "val": transformed_valid_dl}
dataset_sizes = {"train": len(transformed_train_dataset),"val":len(transformed_valid_dataset)}
print(dataset_sizes)

## This function evaluate the loss / optimize  and returns model and the weight of the epoch which has highest accuracy
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        # Each epoch has a training and validation phase
        # In train phase they are settting 2 variable in model class - train() and schedular  = step()
        # In Validation phase setting the model class - eval()
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode
            running_loss = 0.0
            running_corrects = 0
            # Iterate over data.# phase - train or validation
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                labels = labels.type(torch.cuda.LongTensor)
                # zero the parameter gradients
                optimizer.zero_grad()
                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    
                    #print(outputs.shape)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                #print(phase,preds,labels.data)
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            #print(preds[1:10],labels.data[1:10])
            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
        #print()
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

{'train': 18964, 'val': 2000}


In [0]:
# Load Pretrained model

#model_ft = models.resnet18(pretrained=True)
model_ft = models.densenet169(pretrained=True)

# Freeze specific layers of the model

freeze_till('transition2.conv.weight',model_ft)

# num_ftrs = model_ft.fc.in_features # fc for resnet
num_ftrs = model_ft.classifier.in_features
model_ft.classifier = nn.Linear(num_ftrs, 2)
model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss(weight = torch.tensor([1, 1.05]).type(torch.cuda.FloatTensor))

/usr/local/lib/python3.6/dist-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)
Downloading: "https://download.pytorch.org/models/densenet169-b2777c0a.pth" to /root/.torch/models/densenet169-b2777c0a.pth
100%|██████████| 57365526/57365526 [00:04<00:00, 12781536.12it/s]


In [0]:
optimizer_ft = optim.Adam(list(filter(lambda p: p.requires_grad, model_ft.parameters())), lr = 1e-4, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=5, gamma=0.1)
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=10)

Epoch 0/9
----------
train Loss: 0.4272 Acc: 0.8054
val Loss: 0.3270 Acc: 0.8665
Epoch 1/9
----------
train Loss: 0.3088 Acc: 0.8677
val Loss: 0.2107 Acc: 0.9270
Epoch 2/9
----------
train Loss: 0.1560 Acc: 0.9418
val Loss: 0.1544 Acc: 0.9485
Epoch 3/9
----------


KeyboardInterrupt: ignored

In [37]:
!ls

GCP%20Credits%20Request%20Link%20-%20RSNA.txt  stage_1_test_images.zip
Pneumonia-Detection			       stage_1_train_images
sample_data				       stage_1_train_images.zip
stage_1_detailed_class_info.csv		       stage_1_train_labels.csv
stage_1_sample_submission.csv


In [0]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
  def __init__(self, num_classes=2):
      super(ConvNet, self).__init__()
      self.conv1 = nn.Sequential(
          nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=2),
          #nn.BatchNorm2d(16),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2, stride=2))
      self.conv2 = nn.Sequential(
          nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
          #nn.BatchNorm2d(32),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2, stride=2))
      self.conv3 = nn.Sequential(
              nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
              nn.BatchNorm2d(64),
              nn.ReLU(),
              nn.MaxPool2d(kernel_size=2, stride=2))
      self.conv4 = nn.Sequential(
              nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
              nn.BatchNorm2d(128),
              nn.ReLU(),
              nn.MaxPool2d(kernel_size=2, stride=2))
      self.conv5 = nn.Sequential(
              nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
              nn.BatchNorm2d(128),
              nn.ReLU(),
              nn.MaxPool2d(kernel_size=2, stride=2))
      self.conv6 = nn.Sequential(
              nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1),
              nn.BatchNorm2d(64),
              nn.ReLU(),
              nn.MaxPool2d(kernel_size=2, stride=2))

      self.fc1 = nn.Linear(32*32*64, 64)
      self.fc2 = nn.Linear(64, 2)

  def forward(self, x):
      out = self.conv1(x)
      out = self.conv2(out)
      out = self.conv3(out)
      out = self.conv4(out)
      out = self.conv5(out)
      out = self.conv6(out)
      out = out.reshape(out.size(0), -1)
      out = self.fc1(out)
      out = self.fc2(out)
      return out